### 加载模型

In [1]:
from detection.DataGenerator import train_based_self_detection
from detection.Loader.ResNet50Loader import Resnet50Loader
import detection.Spliter
import torch
from torch import nn
print("CODE:loading_resnet50")
model=Resnet50Loader().load()
print("CODE:loading_finished")
device='cuda:2'
back_device='cuda:2'


CODE:loading_resnet50


/home/tianruiming/miniconda3/envs/pytorch/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/tianruiming/miniconda3/envs/pytorch/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


CODE:loading_finished


### 创建量化模型用于测试acc

In [2]:
from torch.quantization.observer import MovingAveragePerChannelMinMaxObserver
observer = MovingAveragePerChannelMinMaxObserver(ch_axis=0).to(device)

class quantiseze_model(nn.Module):
    def __init__(self,model_list):
        super(quantiseze_model,self).__init__()
        self.model_list=model_list
    def forward(self,x):
        x=self.model_list[0](x)
        observer(x)
        scale, zero_point = observer.calculate_qparams()
        # scale=scale.to(device)
        # zero_point=zero_point.to(device)
        x_quantized = torch.quantize_per_channel(x, scales=scale, zero_points=zero_point, axis=0, dtype=torch.qint8)
        x=self.model_list[1](x_quantized.dequantize())
        observer(x)
        scale, zero_point = observer.calculate_qparams()
        # scale=scale.to(device)
        # zero_point=zero_point.to(device)
        x_quantized = torch.quantize_per_channel(x, scales=scale, zero_points=zero_point, axis=0, dtype=torch.qint8)
        x=self.model_list[2](x_quantized.dequantize())
        return x

### 创建测试数据

In [3]:
datamaker=train_based_self_detection(
    model=model,
    device=device,
    no_weight=True
)

input_data,output_label,label,highest_loss,lowest_loss= datamaker.make_data_pid(
        total_number=100,
        batch_size=100,
        learning_rate=1,
        warm_lr=1e-3,
        channel=3,
        dim1=224,
        dim2=224,
        output_size=1000,
        randn_magnification=100,
        confidence=1000000,
        target_acc=0.8

)
print (input_data.dtype)
torch.cuda.empty_cache()

/home/tianruiming/miniconda3/envs/pytorch/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
/home/tianruiming/miniconda3/envs/pytorch/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:240: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
/home/tianruiming/miniconda3/envs/pytorch/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:224: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.

acc: 0.0
loss: 8.444656372070312
acc: 0.0
loss: 8.303812980651855
acc: 0.0
loss: 8.1074800491333
acc: 0.0
loss: 7.924959659576416
acc: 0.0
loss: 7.753497123718262
acc: 0.0
loss: 7.591139793395996
acc: 0.0
loss: 7.436631679534912
acc: 0.01
loss: 7.287731170654297
acc: 0.01
loss: 7.143582820892334
acc: 0.02
loss: 7.003101825714111
acc: 0.02
loss: 6.865771293640137
acc: 0.02
loss: 6.729814529418945
acc: 0.02
loss: 6.595409393310547
acc: 0.03
loss: 6.462466716766357
acc: 0.04
loss: 6.329666614532471
acc: 0.04
loss: 6.196863651275635
acc: 0.04
loss: 6.064444065093994
acc: 0.04
loss: 5.9326171875
acc: 0.04
loss: 5.800970554351807
acc: 0.05
loss: 5.669297218322754
acc: 0.08
loss: 5.538058280944824
acc: 0.1
loss: 5.406550884246826
acc: 0.1
loss: 5.275132179260254
acc: 0.13
loss: 5.1435346603393555
acc: 0.15
loss: 5.011880874633789
acc: 0.16
loss: 4.880213737487793
acc: 0.16
loss: 4.748531818389893
acc: 0.19
loss: 4.6167826652526855
acc: 0.21
loss: 4.48527193069458
acc: 0.21
loss: 4.35493421554

### 初始化搜索器

In [ ]:
cut_step=0.2
searcher=detection.Spliter.Recrusively_reduce_search(
        model=model,
        no_weight=True,
        input_data=input_data,
        output_label=output_label,
        label=label,
        device=device,
        back_device=back_device,
        highest_loss=highest_loss,
        lowest_loss=lowest_loss,
        # local_speed=2.72e10,   #Flops/s
        local_speed=9.6e9,   #Flops/s
        cloud_speed=1.7e13,    #Flops/s
        network_speed=1e7,     #B/s
        acc_cut_point=0.7,
        # q=q,
)
searcher.init(cut_step)

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_op

0it [00:00, ?it/s]

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register count_convNd() f

9it [00:00, 45.31it/s]
7it [00:00, 20.56it/s]
7it [00:00, 29.21it/s]
9it [00:00, 19.89it/s]
7it [00:00, 24.77it/s]
7it [00:00, 27.69it/s]
7it [00:00, 27.69it/s]]
9it [00:00, 13.43it/s]]
7it [00:00, 22.73it/s]]
7it [00:00, 22.23it/s]]
7it [00:00, 22.20it/s]]
7it [00:00, 22.67it/s]]
7it [00:00, 22.30it/s]]
9it [00:01,  7.00it/s]]
7it [00:00,  9.98it/s]]
7it [00:00,  9.97it/s]]
20it [00:07,  1.51it/s]/home/tianruiming/miniconda3/envs/pytorch/lib/python3.10/site-packages/torch/nn/init.py:511: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")
23it [00:07,  2.95it/s]
23it [00:00, 116.10it/s]


SVD_finished
Loss上限: 7.001727104187012
acc上限: 0.0
最长耗时: 49.80096864978824
最短耗时: 25.97664598312157
[0.83, 0.83, 0.83, 0.83, 0.83, 0.84, 0.84, 0.84, 0.84, 0.85, 0.7, 0.7, 0.71, 0.72, 0.8, 0.81, 0.81, 0.85, 0.84, 0.85, 0.84, 0.84, 0.84, 0.85, 0.85, 0.84, 0.84, 0.32, 0.32, 0.34, 0.41, 0.43, 0.43, 0.55, 0.55, 0.55, 0.74, 0.74, 0.74, 0.84, 0.84, 0.84, 0.86, 0.86, 0.86, 0.86, 0.86, 0.86]
[8.02816, 8.02816, 8.02816, 8.02816, 8.02816, 8.02816, 8.02816, 8.02816, 8.02816, 8.02816, 4.01408, 4.01408, 4.01408, 4.01408, 4.01408, 4.01408, 4.01408, 4.01408, 4.01408, 4.01408, 4.01408, 4.01408, 4.01408, 2.00704, 2.00704, 2.00704, 2.00704, 2.00704, 2.00704, 2.00704, 2.00704, 2.00704, 2.00704, 2.00704, 2.00704, 2.00704, 2.00704, 2.00704, 2.00704, 2.00704, 2.00704, 2.00704, 1.00352, 1.00352, 1.00352, 1.00352, 1.00352, 1.00352]


In [5]:
searcher.input_data.shape
upper_bound=searcher.GA_init(50,step=cut_step)

### 寻找最优分割点

In [6]:
upper_num=min(upper_bound)
quantized_network_list=[]
quantized_acc_list=[]
for i in range(2,50):
    model_r,edge_layer_map=searcher.model_reduce([0]*i)
    eA,c,eB=searcher.split(model_r,len(edge_layer_map))
    qm=quantiseze_model([eA,c,eB])
    quantized_network_list.append(searcher.network_evaluate_quantisized(qm))
    acc,loss=searcher.acc_loss_evaluate(qm)
    quantized_acc_list.append(acc)
    # input()
    torch.cuda.empty_cache()
print(quantized_acc_list)
print(quantized_network_list)


    

In [7]:
torch.cuda.empty_cache()